In [152]:
%reset

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
from Crypto.Random import get_random_bytes
from base64 import b16encode
from base64 import b16decode
import os
import json
import hmac
import hashlib

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [153]:
#Initialize Data
data = b"Accnt:0003336669Accnt:0123456789Descr:0000000000000000000000000000000000000000000000000000000000Amount:023456789"
key = get_random_bytes(16)
print(b16encode(key).decode('utf-8'))

205E7F4E5286306EEFABB7625CF6EEFD


# Task A
Encrypt in ECB Mode:

In [154]:
ECB_cipher = AES.new(key, AES.MODE_ECB) #Create a new AES Cypher
ciphertext_bytes = ECB_cipher.encrypt(pad(data, AES.block_size)) #encrypt the padded data
#Encode the cyphertext to 16 bit encoding to have a more clear output
#use the decode function to remove the b'...' in the output
ciphertext = b16encode(ciphertext_bytes).decode('utf-8')
print(ciphertext)

4CDEB5515885FEC7390A48D4F9747BD53A309C3CA17838A5A1FD18A9A433737E9D35FE123067946375B9B59FAD318E1CE9C1DC732606DAA7644898E4D577B90DE9C1DC732606DAA7644898E4D577B90DE9C1DC732606DAA7644898E4D577B90DBD18D9394E2B939E4EA7D9E8C14C0B04FAF69C2C9877EE7A8805287F5B34C232


Decrypt to ensure everything is working properly:

In [155]:
decoded_ciphertext = b16decode(ciphertext) 
decrypted_text = unpad(ECB_cipher.decrypt(decoded_ciphertext), AES.block_size)
print(decrypted_text)

b'Accnt:0003336669Accnt:0123456789Descr:0000000000000000000000000000000000000000000000000000000000Amount:023456789'


Demonstrate Attack:

In [156]:
#Swap the account numbers to reverse the direction of the transfer
modified_ciphertext = ''.join([ciphertext[32:64], ciphertext[0:32], ciphertext[64:]])
#decrypt message:
modified_result = unpad(ECB_cipher.decrypt(b16decode(modified_ciphertext)), AES.block_size)
print(modified_result.decode('utf-8'))

Accnt:0123456789Accnt:0003336669Descr:0000000000000000000000000000000000000000000000000000000000Amount:023456789


# Task B
Encrypt in CBC Mode

In [157]:
iv = os.urandom(16) #initializing vector
CBC_cipher = AES.new(key, AES.MODE_CBC,iv) #Create new AEC Cipher in CBC mode
ciphertext_bytes = CBC_cipher.encrypt(pad(data, AES.block_size)) #Encrypt padded data
#Encode the cyphertext to 16 bit encoding to have a more clear output
#use the decode function to remove the b'...' in the output
CBC_ciphertext = b16encode(ciphertext_bytes).decode('utf-8')
#Save as Json in order to simulate passing the whole thing over the network
json_data = json.dumps({'iv':b16encode(iv).decode('utf-8'), 'ciphertext':CBC_ciphertext})
print(json_data)

{"iv": "BC60BB0C6BD4EC75EAE62683F732C89F", "ciphertext": "77F0FAD593A9DBA008639D01BD9AC24939136D308CDF5C6231B8E3E2014797DBE85EDE39FB4F26B571D78B45DE2CC25C441EB585357F145B1ED3929F4CF2615CBACAA22F4E1C4EF1B8E8730A6EE4D89F607F3391BA16CB837E961755441BB629508777639FC7A7FD179AF69D2F526ADF653E1032E5493CA86E241F417B88CB03"}


Decrypt to ensure everything is working properly:

In [158]:
#Read the data from the json file
b16 = json.loads(json_data)
iv = b16decode(b16['iv'])
#Create a new Cipher for decryption using the same key and iv
CBC_cipher2 = AES.new(key, AES.MODE_CBC, iv)
#Decode, unpad and display the results
CBC_dec_ciphertext = b16decode(b16['ciphertext'])
CBC_decoded_plaintext = unpad(CBC_cipher2.decrypt(CBC_dec_ciphertext),AES.block_size)
print(CBC_decoded_plaintext)

b'Accnt:0003336669Accnt:0123456789Descr:0000000000000000000000000000000000000000000000000000000000Amount:023456789'


XOR Function to be used in attack:

In [159]:
def xor(A, B):
    return hex(int(A, 16) ^ int(B, 16))[2:].upper() 

Demonstrate the attack:

In [160]:
#Attacker can also reda the data from the json file
Oscar_b16 = json.loads(json_data)
Oscar_ciphertext = Oscar_b16['ciphertext']
Oscar_iv = Oscar_b16['iv']
block_to_alter = Oscar_ciphertext[160:192] #block containing amount
byte_to_change = block_to_alter[14:16] #amount

#xor 1st bit of plaintext with the hex value you want in new plaintext
m = xor('0x30', '0x39') #0x30 is decimal 0, 0x39 is decimal 9

#perform bit flipping
new_byte = xor(hex(int(m, 16)), hex(int(byte_to_change, 16)))
while len(new_byte) < 2:
    new_byte = "".join(['0', new_byte])
attacked_ciphertext = ''.join([Oscar_ciphertext[0:174], new_byte, Oscar_ciphertext[176:]]) 

result = json.dumps({'iv':Oscar_iv, 'ciphertext':attacked_ciphertext})

print(result)

{"iv": "BC60BB0C6BD4EC75EAE62683F732C89F", "ciphertext": "77F0FAD593A9DBA008639D01BD9AC24939136D308CDF5C6231B8E3E2014797DBE85EDE39FB4F26B571D78B45DE2CC25C441EB585357F145B1ED3929F4CF2615CBACAA22F4E1C4EF1B8E8730A6EE4D89F607F3391BA16CB8A7E961755441BB629508777639FC7A7FD179AF69D2F526ADF653E1032E5493CA86E241F417B88CB03"}


Decode the attack:

In [161]:
#Create a new Cipher for decryption using the same key and iv
CBC_cipher_Oscar = AES.new(key, AES.MODE_CBC,b16decode(Oscar_iv))
#Decode, unpad and display the results
CBC_dec_ciphertext = b16decode(attacked_ciphertext)
CBC_decoded_plaintext = unpad(CBC_cipher_Oscar.decrypt(CBC_dec_ciphertext),AES.block_size)
print(CBC_decoded_plaintext)

b'Accnt:0003336669Accnt:0123456789Descr:000000000000000000000000000000000000000000\xbbs\x14\x81\x92xC\x975\xbfb\xdf\xaa\\\xc8DAmount:923456789'


# Task C
Set up function to compare MACs

In [162]:
def compare_mac(a, b):
    a = a[1:] 
    b = b[1:]
    different = 0
    
    for x, y in zip(a, b):
        different |= x ^ y
    return different == 0

Encrypt messag in CBC mode but add MAC with HMAC-SHA256

In [163]:
hmac_key = get_random_bytes(16)
plaintext = pad(data, AES.block_size)
iv_bytes = get_random_bytes(AES.block_size)
HMAC_cipher = AES.new(key, AES.MODE_CBC, iv_bytes)
encrypted_data = HMAC_cipher.encrypt(plaintext)
iv = iv_bytes + encrypted_data #secret prefix mac
signature = hmac.new(hmac_key, iv, hashlib.sha256).digest()
print('Encrypted Data:' + b16encode(encrypted_data).decode('utf-8'))
print('IV: ' + b16encode(iv_bytes).decode('utf-8'))
print('Signature: ' + b16encode(signature).decode('utf-8'))

Encrypted Data:176588156FE4A6C27BBA3AFA5B4C769E2B070CCD2BBA766C6BAD0783EA5982125EE9ED55298AA6A9BED32AC2B28C20FE35B0D6585DE2CE3F9493B5D0683090E1EEC665EFDC7B7175DB68CC6F70C5F7029145FFD2A117E44CF2F7FF9A359C9F0D438E2366CAC8575ED9CBD0AEF72C16D1DB6D37749E9A894940A94E3945E96A7C
IV: D4765BBD0E0B5CC66790A39B8E1AC3CB
Signature: 2702B38C869BADB83920DF8D4138FE307D43C945FAFA48B3AB574456FA16A8E8


Verify signature and decrypt if message hasn't been tampered with

In [164]:
new_hmac = hmac.new(hmac_key, iv, hashlib.sha256).digest()
if not compare_mac(new_hmac, signature):
 	print("Incorrect decryption")
cipher = AES.new(key, AES.MODE_CBC, iv_bytes)
dec_plaintext = cipher.decrypt(encrypted_data)
print(unpad(dec_plaintext, AES.block_size).decode('utf-8'))

Accnt:0003336669Accnt:0123456789Descr:0000000000000000000000000000000000000000000000000000000000Amount:023456789
